In [2]:
pip install langChain OpenAI faiss-cpu tiktoken


  Obtaining dependency information for langChain from https://files.pythonhosted.org/packages/49/09/72630413a7ded27684e33392a0ff52ff1c8ea6749fee641319e75f82072b/langchain-0.3.7-py3-none-any.whl.metadata
  Using cached langchain-0.3.7-py3-none-any.whl.metadata (7.1 kB)
  Obtaining dependency information for OpenAI from https://files.pythonhosted.org/packages/77/85/e7adeee84edd24c6cc119b2ccaaacd9579c6a2c7f72d05e936ea6b33594e/openai-1.54.3-py3-none-any.whl.metadata
  Using cached openai-1.54.3-py3-none-any.whl.metadata (24 kB)
  Obtaining dependency information for faiss-cpu from https://files.pythonhosted.org/packages/51/d6/4228ddef6324abb7b6a62cf0dd72938147234770b1f04adcb23fa1a424bb/faiss_cpu-1.9.0-cp312-cp312-win_amd64.whl.metadata
  Using cached faiss_cpu-1.9.0-cp312-cp312-win_amd64.whl.metadata (4.5 kB)
  Obtaining dependency information for tiktoken from https://files.pythonhosted.org/packages/45/e2/39d4aa02a52bba73b2cd21ba4533c84425ff8786cc63c511d68c8897376e/tiktoken-0.8.0-cp312-c


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
pip install python-dotenv

  Obtaining dependency information for python-dotenv from https://files.pythonhosted.org/packages/6a/3e/b68c118422ec867fa7ab88444e1274aa40681c606d59ac27de5a5588f082/python_dotenv-1.0.1-py3-none-any.whl.metadata
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
pip install langchain-community

  Obtaining dependency information for langchain-community from https://files.pythonhosted.org/packages/82/c7/b6d7cb286e93f85b4882088ce530cc7d55a5683c9390d7fc1c0dbde846e2/langchain_community-0.3.5-py3-none-any.whl.metadata
  Obtaining dependency information for SQLAlchemy<2.0.36,>=1.4 from https://files.pythonhosted.org/packages/60/63/a3cef44a52979169d884f3583d0640e64b3c28122c096474a1d7cfcaf1f3/SQLAlchemy-2.0.35-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for dataclasses-json<0.7,>=0.5.7 from https://files.pythonhosted.org/packages/c3/be/d0d44e092656fe7a06b55e6103cbce807cdbdee17884a5367c68c9860853/dataclasses_json-0.6.7-py3-none-any.whl.metadata
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Obtaining dependency information for httpx-sse<0.5.0,>=0.4.0 from https://files.pythonhosted.org/packages/e1/9b/a181f281f65d776426002f330c31849b86b31fc9d848db62e16f03ff739f/httpx_sse-0.4.0-py3-none-any.whl.metadata
  Obtaining dependency informat


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
pip install langchain-openai

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: langchain-openai in f:\kelaniya se\lectures\level 4\semester 2\seng 43216 (research)\rag youtube\rag-be\venv\lib\site-packages (0.2.6)




[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
pip install pypdf

  Obtaining dependency information for pypdf from https://files.pythonhosted.org/packages/04/fc/6f52588ac1cb4400a7804ef88d0d4e00cfe57a7ac6793ec3b00de5a8758b/pypdf-5.1.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/298.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/298.0 kB ? eta -:--:--
   - -------------------------------------- 10.2/298.0 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/298.0 kB 330.3 kB/s eta 0:00:01
   ---- ---------------------------------- 30.7/298.0 kB 330.3 kB/s eta 0:00:01
   ----- --------------------------------- 41.0/298.0 kB 219.4 kB/s eta 0:00:02
   -------- ------------------------------ 61.4/298.0 kB 252.2 kB/s eta 0:00:01
   ---------- ---------------------------- 81.9/298.0 kB 306.3 kB/s eta 0:00:01
   ------------ -------------------------- 92.2/298.0 kB 309.1 kB/s eta 0:00:01
   -------------- ----------------------- 112.6/298.0 kB 312.2 kB/s eta 0:00:01
   --------------- ------------


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from dotenv import load_dotenv
import os

load_dotenv() 
api_key = os.getenv("OPEN_AI_KEY")

In [21]:
import openai, langchain, faiss, tiktoken, pypdf

from langchain_community.document_loaders import DirectoryLoader, TextLoader, UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

from langchain.llms import OpenAI

from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA


### import own pdf file

In [22]:
loader = DirectoryLoader("data" , glob="./*.pdf" ,loader_cls=PyPDFLoader)
document = loader.load() 

## split the text 

In [23]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(document)

In [28]:
type(texts)

list

In [27]:
len(texts)

1045

In [30]:
texts[100]

Document(metadata={'source': 'data\\David Tarnoff.pdf', 'page': 32}, page_content='Chapter 1:  Digital Signals and Systems    9 \n \nAlternatively, multiple lines can be combined into a more abstract \nrepresentation such as the one shown in Figure 1-10. \n \n \n \n \n \n \n \n \n \nFigure 1-10   Alternate Representation of Multiple Digital Signals \nTwo horizontal lines, one at a logic 1 level and one at a logic 0 level \nindicate constant signals from all of the lines represented. A single \nhorizontal line running approximately between logic 1 and logic 0 \nmeans that the signals are not sending any data. This is different from \nan "off" or logic 0 in that a logic 0 indicates a number while no data \nmeans that the device transmitting the data is not available. Hash marks \nindicate invalid or changing data. This could mean that one or all of the \nsignals are changing their values, or that due to the nature of the \nelectronics, the values of the data signals cannot be predicted. 

In [31]:
texts[100].metadata

{'source': 'data\\David Tarnoff.pdf', 'page': 32}

In [32]:
texts[100].metadata['source']

'data\\David Tarnoff.pdf'

### OpenAi Embedding and VectorDB

In [33]:
embeddings = OpenAIEmbeddings(api_key=api_key)

In [34]:
# create vector from text

docsearch = FAISS.from_documents(texts, embeddings)

In [35]:
query1 = "what are edges and pulses"

In [36]:
answer = docsearch.similarity_search(query1)

In [41]:
print(answer[3].page_content)

10   Computer Organization and Design Fundamentals 
 
computer (momentarily pressed, then released) or the button used to 
initialize synchronization between a PDA and a computer. 
 
 
Figure 1-11   Digital Transition Definitions 
There are two types of pulses. The first is called a positive-going 
pulse, and it has an idle state of logic 0 with a short pulse to logic 1. 
The other one, a negative-going pulse, has an idle state of logic 1 with 
a short pulse to logic 0. Both of these signals are shown in Figure 1-12. 
 
 
 
Figure 1-12   Pulse Waveforms 
1.6.3 Non-Periodic Pulse Trains 
Some digital signals such as the data wires of an Ethernet link or the 
data and address lines of a memory interface do not have a 
characteristic pattern in their changes between logic 1 and logic 0. 
These are called non-periodic pulse trains. 
 
 
 
Figure 1-13   Non-Periodic Pulse Train 
Like music, the duration of the notes or the spaces between the notes


In [ ]:
answer_score = docsearch.similarity_search_with_score(query1) # closer to 0 is better

In [43]:
answer_score[0]

(Document(metadata={'source': 'data\\David Tarnoff.pdf', 'page': 4}, page_content="1.6.1 Edges ................................................................................. 9 \n  1.6.2 Pulses................................................................................. 9 \n  1.6.3 Non-Periodic Pulse Trains .............................................. 10 \n  1.6.4 Periodic Pulse Trains....................................................... 11 \n  1.6.5 Pulse-Width Modulation ................................................. 13 \n 1.7 Unit Prefixes ............................................................................ 15 \n 1.8 What's Next? ............................................................................ 16 \n Problems......................................................................................... 16 \n \nChapter Two: Numbering Systems .................................................. 17 \n 2.1 Unsigned Binary Counting.................................

### import chat

In [45]:
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA

In [51]:
## setup the llm model

llm = OpenAI(temperature=0, api_key=api_key)

### get answers from your own docs 

In [76]:
# helper function to process the response from the QA chain
# and isloate result and source docs and page number

def parse_response(response):
    print(response['result'])
    print('\n\nSource:')
    for source_name in response['source_documents']:
        print(source_name.metadata['source'], "page #:", source_name.metadata['page'])

In [57]:
# setup the retriever on the faiss vector store
# make sure to set include_metadata = True

retriever = docsearch.as_retriever(include_metadata=True, metadata_key='source')

In [69]:
# setup the RetrieverQA chain with the retriever 
# make sure to return_source_documents = True

qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type="stuff",
                                       retriever=retriever,
                                       return_source_documents=True)

In [70]:
query = "what are edges and pulses"

In [71]:
response = qa_chain(query)

In [72]:
type(response)

dict

In [73]:
response

{'query': 'what are edges and pulses',
 'result': ' Edges and pulses are types of digital signals that occur in binary systems. Edges refer to the transition from one logic value to another, while pulses occur when a signal changes from one value to the other for a short period before returning to its original value. There are two types of pulses: positive-going and negative-going. Non-periodic pulse trains are digital signals that do not have a characteristic pattern in their changes between logic 1 and logic 0.',
 'source_documents': [Document(metadata={'source': 'data\\David Tarnoff.pdf', 'page': 4}, page_content="1.6.1 Edges ................................................................................. 9 \n  1.6.2 Pulses................................................................................. 9 \n  1.6.3 Non-Periodic Pulse Trains .............................................. 10 \n  1.6.4 Periodic Pulse Trains....................................................... 11 \n 

In [78]:
parse_response(response)

 Edges and pulses are types of digital signals that occur in binary systems. Edges refer to the transition from one logic value to another, while pulses occur when a signal changes from one value to the other for a short period before returning to its original value. There are two types of pulses: positive-going and negative-going. Non-periodic pulse trains are digital signals that do not have a characteristic pattern in their changes between logic 1 and logic 0.


Source:
data\David Tarnoff.pdf page #: 4
data\David Tarnoff.pdf page #: 33
data\David Tarnoff.pdf page #: 32
data\David Tarnoff.pdf page #: 33


In [75]:
response['source_documents']

[Document(metadata={'source': 'data\\David Tarnoff.pdf', 'page': 4}, page_content="1.6.1 Edges ................................................................................. 9 \n  1.6.2 Pulses................................................................................. 9 \n  1.6.3 Non-Periodic Pulse Trains .............................................. 10 \n  1.6.4 Periodic Pulse Trains....................................................... 11 \n  1.6.5 Pulse-Width Modulation ................................................. 13 \n 1.7 Unit Prefixes ............................................................................ 15 \n 1.8 What's Next? ............................................................................ 16 \n Problems......................................................................................... 16 \n \nChapter Two: Numbering Systems .................................................. 17 \n 2.1 Unsigned Binary Counting.................................

### use Vector agent

In [91]:
# import the dependencies

from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo
)

In [80]:
# set up the vectorstore info

vectorstore_info = VectorStoreInfo(
    name="pdf_vectorstore",
    description="pdf vectorstore",
    vectorstore=docsearch,
)

In [102]:
# Setup the VectorStoreToolkit and VectorStoreAgent

toolkit = VectorStoreToolkit(llm=llm, vectorstore_info=vectorstore_info)
agent_executor = create_vectorstore_agent(llm=llm,
                                          toolkit=toolkit,
                                          verbose=False)

In [104]:
# Add the string to ask for source

query = query + " List the sources."
print(query)

what are edges and pulsesList the sourcesList the sources List the sources List the sources List the sources. List the sources. List the sources. List the sources. List the sources. List the sources.


In [113]:
# run the agent

response = agent_executor.run(query)

In [ ]:
type(response)

str

In [ ]:
response

'Edges are transitions from one binary value to another, while pulses are short periods of a signal changing from one value to another. Non-periodic pulse trains are digital signals without a characteristic pattern in their changes between logic 1 and logic 0.'